In [1]:
# Imports
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
from selenium import webdriver
from time import sleep
import random

In [32]:
# Declaring URL, WebDriver location, minimum request and sleep interval
url = "https://www.indeed.com.sg/jobs?q='data+analyst'&l=Singapore&start={}&limit=50"
chromedriver = "chromedriver.exe"
minreq = 1200
sleepy = random.randint(3, 7)

In [33]:
# Execution of URL in WebDriver and retrieve source
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(url.format(0))
sleep(sleepy)
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [34]:
# Indeed Code Begins Here #
job_id = []
job_title = []
coy_name = []
coy_rating = []
review_count = []
job_location = []
alternate_source = []
min_salary = []
max_salary = []
days_posted = []
job_desc = []
job_type = []
job_url = []
job_count = 0
jip_count = 0
is_responsive = []
is_contract = []
is_internship = []
is_temporary = []
is_permanent = []

while job_count < minreq:
    list_of_jobs = soup.findAll("div", ["row result clickcard", "lastRow row result clickcard", "row sjlast result clickcard"])
    if jip_count < len(list_of_jobs):
        for no, item in enumerate(list_of_jobs):
            # ID
            job_id.append(item["id"])
            # Job Title
            job_title.append(item.findAll("a", {"data-tn-element":"jobTitle"})[0]["title"])
            # Company Name
            company_name_var = item.findAll("span", {"class":"company"})
            if company_name_var:
                coy_name.append(company_name_var[0].text.split("\n        ")[1])
            else:
                coy_name.append("Unknown")
            # Reviews
            number_reviews_var = item.findAll("span", "slNoUnderline")
            if len(number_reviews_var) > 0:
                review_count.append(int("".join(number_reviews_var[0].text.split(" ")[0].split(","))))
            else:
                review_count.append(0)
            # Location
            location_var = item.findAll(["span", "div"], "location")
            job_location.append(location_var[0].text)
            # Alternative Source (if any?)
            alternate_source_var = item.findAll("span", "result-link-source")
            if alternate_source_var:
                alternate_source.append(alternate_source_var[0].text)
            else:
                alternate_source.append("Unknown")
            # Salary
            salary_var = item.findAll("span", "no-wrap")
            if len(salary_var) > 0:
                if len(salary_var[0].text.split("$")) > 2:  # If there are 2 numbers given
                    minimum = int("".join(salary_var[0].text.split("$")[1].split(" ")[0].split(",")))
                    maximum = int("".join(salary_var[0].text.split("$")[2].split(" ")[0].split(",")))
                    min_salary.append(minimum)
                    max_salary.append(maximum)
                else:  # If there is only 1 number given
                    minimum = int("".join(salary_var[0].text.split("$")[1].split(" ")[0].split(",")))
                    min_salary.append(minimum)
                    max_salary.append(minimum)
            else:  # If there are 0 number given
                min_salary.append(0)
                max_salary.append(0)
            # Responsive Employer?
            if item.findAll("div", "serp-ResponsiveEmployer-heading"):
                is_responsive.append(1)
            else:
                is_responsive.append(0)
                
            # Next Level Stuff
            button = driver.find_element_by_xpath('//*[@id="'+ item["id"] +'"]')
            button.click()
            driver.switch_to_window(driver.window_handles[1])
            soup_level2 = BeautifulSoup(driver.page_source, 'lxml')
            # coy rating
            coy_rating_var = soup_level2.findAll("div", "icl-Ratings-starsCountWrapper")
            if coy_rating_var:
                coy_rating.append(float(coy_rating_var[0]["aria-label"].split(" out")[0]))
            else:
                coy_rating.append(float(0.0))
            # job type
            job_type_var = soup_level2.findAll("div", "jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs")
            list1 = ["Contract", "Internship", "Temporary", "Permanent"]
            list2 = [is_contract, is_internship, is_temporary, is_permanent]
            if job_type_var:
                job_type.append(job_type_var[0].text)
                for num, pos in enumerate(list1):
                    if pos in job_type_var[0].text.split(", "):
                        list2[num].append(1)
                    else:
                        list2[num].append(0)
            else:
                job_type.append("Unknown")
                for num, pos in enumerate(list1):
                    list2[num].append(0)
            # days
            days_posted_var = soup_level2.findAll("div", "jobsearch-JobMetadataFooter")
            if days_posted_var:
                text = days_posted_var[0].text.split(" - ")
                if "ago" in text[0]:
                    days_posted.append(text[0].split(" ")[0])
                elif "ago" in text[1]: # special case if some company got weird names
                    days_posted.append(text[1].split(" ")[0])
                else:
                    days_posted.append(text[2].split(" ")[0])
            else:
                days_posted.append("None")
            # job description
            job_description_var = soup_level2.findAll("div", "jobsearch-JobComponent-description icl-u-xs-mt--md")
            #if len(job_description_var[0].findAll(["p", "ul"])) > 0:
            #    job_desc.append(" ".join([a.text for a in job_description_var[0].findAll(["p", "li"])]))
            if job_description_var:
                job_desc.append(job_description_var[0].text)
            else:
                job_desc.append("None")
            # job url
            job_url.append(driver.current_url)
            # END OF SCRAPING
            sleep(sleepy)
            driver.close()
            driver.switch_to_window(driver.window_handles[0])
            job_count+=1
            jip_count+=1
            print(job_count)
            if job_count == minreq:
                break
        # end of for loop
    else:
        if (soup.findAll("span", "np")[-1].text == "Next\xa0»"):
            button = driver.find_element_by_partial_link_text('»')
            button.click()
            sleep(sleepy)
            soup = BeautifulSoup(driver.page_source, 'lxml')
            jip_count = 0
            try:
                button = driver.find_element_by_link_text("No, thanks")
                button.click()
            except:
                continue
        else:
            driver.close()
            print("Done")
            break
else:
    driver.close()
    print("Done")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [35]:
# Formatting the DataFrame before saving to CSV
df = pd.DataFrame([job_id, job_url, job_title, job_type, coy_name, coy_rating, review_count, job_location, alternate_source, min_salary, max_salary, days_posted, is_responsive, is_contract, is_internship, is_temporary, is_permanent, job_desc],
             index=["Job ID", "URL", "Job Title", "Job Type", "Company Name", "Company Rating", "Number of Reviews", "Job Location", "Alternate Source", "Min Salary", "Max Salary", "Days Posted", "Responsive", "Contract", "Internship", "Temporary", "Permanent", "Job Description"]).T
df

,Job ID,URL,Job Title,Job Type,Company Name,Company Rating,Number of Reviews,Job Location,Alternate Source,Min Salary,Max Salary,Days Posted,Responsive,Contract,Internship,Temporary,Permanent,Job Description
0,pj_b4fd692261af3cdc,https://www.indeed.com.sg/viewjob?jk=b4fd69226...,Data Analyst,Unknown,Wells Fargo,3.8,29807,Singapore,Unknown,0,0,30+,0,0,0,0,0,About Wells Fargo\nWells Fargo & Company (NYSE...
1,pj_8b579131c728614c,https://www.indeed.com.sg/viewjob?jk=8b579131c...,Customer Insights Analyst,Unknown,Sparkline,0,0,Singapore,Unknown,0,0,28,0,0,0,0,0,Description\nThis job opportunity is listed on...
2,pj_b2b7f1fd74fc0fdf,https://www.indeed.com.sg/viewjob?jk=b2b7f1fd7...,Information Technology (IT) – Business Analyst,Unknown,Procter and Gamble,4.2,4781,Central Singapore,Unknown,0,0,22,0,0,0,0,0,Senior Business analysts at P&G drive better d...
3,pj_8a4ff0f7e8db2271,https://www.indeed.com.sg/viewjob?jk=8a4ff0f7e...,"AML Analyst, Data Solutions (1-year contract t...",Permanent,Dow Jones,3.9,180,Singapore,Unknown,0,0,30+,0,0,0,0,1,PermanentJob Description:\nThe Data Solutions ...
4,p_b909911f6bf9c5ff,https://www.indeed.com.sg/viewjob?jk=b909911f6...,Data Analyst intern,Internship,AXA,3.8,778,Singapore,Unknown,0,0,17,0,0,1,0,0,InternshipYou will be part of the Investment D...
5,p_e7fc3ddede15d4d0,https://www.indeed.com.sg/viewjob?jk=e7fc3dded...,Data Analyst (Marketing),Unknown,Grab Taxi,4.1,101,Singapore,Unknown,0,0,30+,0,0,0,0,0,"Get to know the Role:\nTo design, set up, and ..."
6,p_055c90fd2cbdff23,https://www.indeed.com.sg/viewjob?jk=055c90fd2...,Data Operations Analyst – ICIS – Singapore,Unknown,Reed Business Information,4,61,Singapore,RELX Group,0,0,8,0,0,0,0,0,Data Operations Analyst – ICIS – Singapore\n\n...
7,p_149398acbb5466bd,https://www.indeed.com.sg/viewjob?jk=149398acb...,Data Analyst,Unknown,AIA,0,4,Singapore,Unknown,0,0,29,0,0,0,0,0,Data Analyst\n\nThe Data Analyst (DA) will be ...
8,p_4fd8b2c2587861aa,https://www.indeed.com.sg/viewjob?jk=4fd8b2c25...,"Analyst, Data Analytics, Garena",Unknown,Garena,3.6,5,Singapore,Unknown,0,0,30+,0,0,0,0,0,Garena is looking for an Analyst to improve bu...
9,p_332e2ff8c6e6f73a,https://www.indeed.com.sg/viewjob?jk=332e2ff8c...,Data Analyst,Unknown,Wego Pte Ltd,0,0,Singapore,Unknown,0,0,30+,0,0,0,0,0,As part of the Data Science and Analytics team...


In [31]:
# Saving the DataFrame as CSV
df.to_csv("indeed_dataanalyst.csv")